https://colab.research.google.com/drive/1C7-7qZrYu-7e36OhfitR-Rop_E94O9Vz#scrollTo=REGWp7ArGEi0

In [ ]:
import torch
from collections import Counter



---



x1,y1,x2,y2

(x1,y1): Координата верхнего левого угла BBOX (может быть нормализована)

(x2,y2): Координата нижнего правого угла BBOX (возможно, была нормализована)

In [ ]:
boxes_labels = torch.tensor([[30,40,80,80],[20,20,70,80]])
boxes_preds = torch.tensor([[50,35,70,105],[20,25,75,70]])

**функция рабочая (на листике считал)!**

In [ ]:
def insert_over_union(boxes_preds,boxes_labels):
#x1,y1,x2,y2
#(x1,y1): Координата нижнего левого угла BBOX (может быть нормализована)
#(x2,y2): Координата верхнего правого угла BBOX (возможно, была нормализована)
# !!!!!!!!! если перепутать, будет на выходе '0'
    box1_x1=boxes_preds[...,0:1]
    box1_y1=boxes_preds[...,1:2]
    box1_x2=boxes_preds[...,2:3]
    box1_y2=boxes_preds[...,3:4]#shape:[N,1]

    box2_x1=boxes_labels[...,0:1]
    box2_y1=boxes_labels[...,1:2]
    box2_x2=boxes_labels[...,2:3]
    box2_y2=boxes_labels[...,3:4]

    x1=torch.max(box1_x1,box2_x1)
    y1=torch.max(box1_y1,box2_y1)
    x2=torch.min(box1_x2,box2_x2)
    y2=torch.min(box1_y2,box2_y2)


    #Calculate the area of intersection area
    intersection=(x2-x1).clamp(0)*(y2-y1).clamp(0)

    box1_area=abs((box1_x2-box1_x1)*(box1_y1-box1_y2))
    box2_area=abs((box2_x2-box2_x1)*(box2_y1-box2_y2))

    return intersection/(box1_area+box2_area-intersection+1e-6)

In [ ]:
insert_over_union(boxes_preds,boxes_labels)

tensor([[0.3077],
        [0.6977]])



---



In [ ]:
boxes_labels = [[1,0,1.,30,40,80,80],[2,1,1.,30,80,80,40]]
boxes_preds = [[1,0,0.4,50,35,70,105],[2,0,0.5,50,70,105,35]]

In [ ]:
# этот срабатывает
# Xmin,Ymin,Xmax,Ymax
boxes_labels = [[0,0,1,30,40,80,80],[1,1,1,20,20,70,80]]
boxes_preds = [[0,0,0.4,50,35,70,105],[1,1,0.9,20,25,75,70],[1,1,0.7,5,40,45,60],[1,1,0.1,85,70,105,100]]

In [ ]:
# этот пока не срабатывает
# такие координаты указывал автор кода, но так у меня получаются нули
# Xmin,Ymax,Xmax,Ymin
boxes_labels = [[0,0,1.,30,80,80,40],[1,1,1.,20,80,70,20]]
boxes_preds = [[0,0,0.4,50,105,70,35],[1,1,0.9,20,70,75,25],[1,1,0.7,5,60,45,40],[1,1,0.1,85,100,105,70]]

In [ ]:
num_classes=2
for c in range(num_classes):
  print(c)

0
1


In [ ]:
import torch
from collections import Counter

def mean_average_precision(pred_bboxes, true_boxes, iou_threshold=0.2, num_classes=2):

    #pred_bboxes(list): [[train_idx,class_pred,prob_score,x1,y1,x2,y2], ...]
    print(true_boxes)
    print(pred_bboxes)
    average_precisions=[]#Пустой список для AP
    epsilon=1e-6#коэффициент для избежания ошибок

    #Для каждой категории!!!!!!!!!!!
    for c in range(num_classes):
        print(f'=================================================класс {c}=====================================================================')
        detections=[]#Store bbox predicted for this category
        ground_truths=[]#Storage itself is bbox(GT) of this category

        for detection in pred_bboxes: # список данных для pred (предсказаний)
            if detection[1]==c:
                detections.append(detection)
        # print('------detections--------')
        # print(detections)

        for true_box in true_boxes: # список данных для true (исходных коробок)
            if true_box[1]==c:
                ground_truths.append(true_box)
        # print('------ground_truths--------')
        # print(ground_truths)

        # print(ground_truths)
        #img 0 has 3 bboxes
        #img 1 has 5 bboxes
        #It's like this: amount_bboxes={0:3,1:5}
        #Count the number of real boxes in each picture_ IDX indicates the number of the image to distinguish each image
        amount_bboxes=Counter(gt[0] for gt in ground_truths) # создаем amount_bboxes={0:3,1:5}, для подсчета количества реальных блоков на каждом изображении
        # print('------amount_bboxes--------')
        # print(amount_bboxes)
        for key,val in amount_bboxes.items(): #изменяем amount_bboxes в amount_bboxes={0:torch.tensor([0,0,0]),1:torch.tensor([0,0,0,0,0])}
            amount_bboxes[key]=torch.zeros(val)#Set to 0, which means that none of these real boxes initially matches any prediction box
        print('--------------')
        print(amount_bboxes)

        #Rank the prediction boxes in order of confidence
        detections.sort(key=lambda x:x[2],reverse=True) # сортируем от 1 до 0 по степени уверенности модели в своих предсказаниях
        # print('--------------')
        # print(detections)
        #Initialize TP,FP
        TP=torch.zeros(len(detections)) # создаем нулевой тензор TP
        FP=torch.zeros(len(detections)) # создаем нулевой тензор FP
        # print('------TP---FP-------')
        # print(TP)
        # print(FP)
        #TP+FN это общее количество ящиков GT в текущей категории, которое является фиксированным
        total_true_bboxes=len(ground_truths) # ground_truths -  список данных для true (исходных коробок)
        # print('------total_true_bboxes--------')
        # print(total_true_bboxes)
        #Если в текущей категории нет поля GT, вы можете пропустить его напрямую
        if total_true_bboxes == 0:
            continue

        #For each prediction box, first find all the real boxes in its picture, and then calculate the IoU between the prediction box and each real box. If the IoU threshold is greater than the IoU threshold and the real box does not match other prediction boxes, the prediction result of the prediction box is set as TP, otherwise it is FP
        # Для каждого поля предсказания сначала найдите все реальные поля на его картинке, а затем вычислите долговую расписку между полем предсказания и каждым реальным полем. Если пороговое значение долговой расписки больше, чем пороговое значение долговой расписки,
        # и поле real не соответствует другим полям прогнозирования, результат прогнозирования для поля прогнозирования устанавливается как TP, в противном случае это FP
        # print(ground_truths)
        # print(detections)
        for detection_idx,detection in enumerate(detections):
            # print('-------detection_idx,detection--------')
            # print(detection_idx)
            # print(detection)
            #При расчете долговой расписки это может быть сделано только на одной картинке, а не между разными картинками
            #Номер картинки имеет 0-е измерение
            #Итак, функция следующего кода такова: найдите все реальные ячейки на изображении текущего обнаружения ячейки прогнозирования и используйте их для вычисления долговой расписки
            ground_truth_img=[bbox for bbox in ground_truths if bbox[0]==detection[0]] # список, которое выдергивает данные из ground_truths для 1 картинки, если индексы с данными в detection совпадают
            # print('------ground_truth_img--------')
            # print(ground_truth_img)
            num_gts=len(ground_truth_img) # длина списка коробок true

            best_iou=0
            for idx,gt in enumerate(ground_truth_img): # для каждой истинной рамки ...
                #Вычислите IoU текущего обнаружения блока прогнозирования и каждого реального блока на его изображении
                iou=insert_over_union(torch.tensor(detection[3:]),torch.tensor(gt[3:]))
                print('----------------------------iou--------')
                print(iou)
                # print(best_iou)
                if iou >best_iou: # ищем лучшее значение IoU
                    best_iou=iou
                    best_gt_idx=idx # индекс лучшего значения для класса
                    # print('----------------------------------------------best_iou--------')
                    # print(best_iou)
                    # print(best_gt_idx)
                # else: best_iou
            if best_iou>iou_threshold: # отсекаем предсказания с IoU ниже порога
                #The detection[0] here is amount_ A key of bboxes, representing the number of the picture, best_gt_idx is the subscript of the real box in the value corresponding to the key
                #Обнаружение[0] здесь - это amount_ Ключ bboxes, представляющий номер изображения, best_gt_idx - это индекс реального поля в значении, соответствующем ключу
                #print(amount_bboxes[detection[0]])
                if amount_bboxes[detection[0]][best_gt_idx]==0:#Only the unoccupied real box can be used, and 0 means unoccupied (occupied: the real box matches a prediction box [both IoU are greater than the set IoU threshold])
                #Можно использовать только незанятое реальное поле, а 0 означает незанятое (занято: реальное поле совпадает с полем прогнозирования [обе IoU превышают установленный порог долговых обязательств])
                    TP[detection_idx]=1#The prediction box is TP
                    amount_bboxes[detection[0]][best_gt_idx]=1#Mark the real box as used and cannot be used for other prediction boxes. Because a prediction box can only correspond to one real box at most (at most: when IoU is less than IoU threshold, the prediction box has no corresponding real box)
                    #Отметьте поле real как используемое и не может быть использовано для других полей прогнозирования. Поскольку поле прогнозирования может соответствовать не более чем одному реальному полю (самое большее: когда долговая расписка меньше порогового значения, поле прогнозирования не имеет соответствующего реального поля)
                else:
                    FP[detection_idx]=1#Although the IoU between the prediction box and one of the real boxes is greater than the IoU threshold, the real box has already matched with other prediction boxes, so the prediction box is FP
            else:
                FP[detection_idx]=1#The IoU between the prediction box and each box in the real box is less than the IoU threshold, so the prediction box is directly FP
            print('------FP--TP---------')
            print(FP)
            print(TP)
        TP_cumsum=torch.cumsum(TP,dim=0)
        FP_cumsum=torch.cumsum(FP,dim=0)
        print('------TP_cumsum--FP_cumsum---------')
        print(TP_cumsum)
        print(FP_cumsum)
        #apply a formula
        recalls=TP_cumsum/(total_true_bboxes+epsilon)
        precisions=torch.divide(TP_cumsum,(TP_cumsum+FP_cumsum+epsilon))
        print('------precisions--recalls---------')
        print(precisions)
        print(recalls)
        #Add the point [0,1]
        precisions=torch.cat((torch.tensor([1]),precisions))
        recalls=torch.cat((torch.tensor([0]),recalls))
        print('------precisions--recalls---------')
        print(precisions)
        print(recalls)
        #Using trapz to calculate AP
        average_precisions.append(torch.trapz(precisions,recalls))
        print('------average_precisions---------')
        print(average_precisions)
    return sum(average_precisions)/len(average_precisions), average_precisions


def insert_over_union(boxes_preds,boxes_labels):
#x1,y1,x2,y2
#(x1,y1): Координата нижнего левого угла BBOX (может быть нормализована)
#(x2,y2): Координата верхнего правого угла BBOX (возможно, была нормализована)
# !!!!!!!!! если перепутать, будет на выходе '0'
    box1_x1=boxes_preds[...,0:1]
    box1_y1=boxes_preds[...,1:2]
    box1_x2=boxes_preds[...,2:3]
    box1_y2=boxes_preds[...,3:4]#shape:[N,1]

    box2_x1=boxes_labels[...,0:1]
    box2_y1=boxes_labels[...,1:2]
    box2_x2=boxes_labels[...,2:3]
    box2_y2=boxes_labels[...,3:4]

    x1=torch.max(box1_x1,box2_x1)
    y1=torch.max(box1_y1,box2_y1)
    x2=torch.min(box1_x2,box2_x2)
    y2=torch.min(box1_y2,box2_y2)


    #Calculate the area of intersection area
    intersection=(x2-x1).clamp(0)*(y2-y1).clamp(0)

    box1_area=abs((box1_x2-box1_x1)*(box1_y1-box1_y2))
    box2_area=abs((box2_x2-box2_x1)*(box2_y1-box2_y2))
    # print(box1_area)
    # print(box2_area)
    return intersection/(box1_area+box2_area-intersection+1e-6)

In [ ]:
mean_average_precision(boxes_preds,boxes_labels)

[[0, 0, 1, 30, 40, 80, 80], [1, 1, 1, 20, 20, 70, 80]]
[[0, 0, 0.4, 50, 35, 70, 105], [1, 1, 0.9, 20, 25, 75, 70], [1, 1, 0.7, 5, 40, 45, 60], [1, 1, 0.1, 85, 70, 105, 100]]
=================================================класс 0=====================================================================
--------------
Counter({0: tensor([0.])})
----------------------------iou--------
tensor([0.3077])
------FP--TP---------
tensor([0.])
tensor([1.])
------TP_cumsum--FP_cumsum---------
tensor([1.])
tensor([0.])
------precisions--recalls---------
tensor([1.0000])
tensor([1.0000])
------precisions--recalls---------
tensor([1.0000, 1.0000])
tensor([0.0000, 1.0000])
------average_precisions---------
[tensor(1.0000)]
=================================================класс 1=====================================================================
--------------
Counter({1: tensor([0.])})
----------------------------iou--------
tensor([0.6977])
------FP--TP---------
tensor([0., 0., 0.])
tensor([1., 0., 0.

(tensor(1.0000), [tensor(1.0000), tensor(1.0000)])

#**Пробуем выводить по центральной точке и ширине и высоте**

In [ ]:
def intersection_over_union(boxes_preds, boxes_labels, box_format='midpoint'):

	"""
         boxes_preds (tensor): predictive boundary frame (BATCH_SIZE, 4)
         Boxes_labels (tensor): real boundary frame (BATCH_SIZE, 4)
         box_format (str): Midpoint/Corners, the shape of bbox is (x, y, w, h) or (x1, y1, x2, y2)
	"""

    if box_format == "midpoint":
        box1_x1 = boxes_preds[..., 0:1] - boxes_preds[..., 2:3] / 2
        box1_y1 = boxes_preds[..., 1:2] - boxes_preds[..., 3:4] / 2
        box1_x2 = boxes_preds[..., 0:1] + boxes_preds[..., 2:3] / 2
        box1_y2 = boxes_preds[..., 1:2] + boxes_preds[..., 3:4] / 2
        box2_x1 = boxes_labels[..., 0:1] - boxes_labels[..., 2:3] / 2
        box2_y1 = boxes_labels[..., 1:2] - boxes_labels[..., 3:4] / 2
        box2_x2 = boxes_labels[..., 0:1] + boxes_labels[..., 2:3] / 2
        box2_y2 = boxes_labels[..., 1:2] + boxes_labels[..., 3:4] / 2

    if box_format == "corners":
        box1_x1 = boxes_preds[..., 0:1]
        box1_y1 = boxes_preds[..., 1:2]
        box1_x2 = boxes_preds[..., 2:3]
        box1_y2 = boxes_preds[..., 3:4]  # (N, 1)
        box2_x1 = boxes_labels[..., 0:1]
        box2_y1 = boxes_labels[..., 1:2]
        box2_x2 = boxes_labels[..., 2:3]
        box2_y2 = boxes_labels[..., 3:4]

    x1 = torch.max(box1_x1, box2_x1)
    y1 = torch.max(box1_y1, box2_y1)
    x2 = torch.min(box1_x2, box2_x2)
    y2 = torch.min(box1_y2, box2_y2)

	# .clamp (0) for the situation where Box did not intersect
    intersection = (x2 - x1).clamp(0) * (y2 - y1).clamp(0)

    box1_area = abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
    box2_area = abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1))

    return intersection / (box1_area + box2_area - intersection + 1e-6)

In [ ]:
import torch
from collections import Counter

from iou import intersection_over_union

def mean_average_precision(
    pred_boxes, true_boxes, iou_threshold=0.5, box_format="midpoint", num_classes=20
):
    """
    Calculates mean average precision

    Parameters:
        pred_boxes (list): list of lists containing all bboxes with each bboxes
        specified as [train_idx, class_prediction, prob_score, x1, y1, x2, y2]
        true_boxes (list): Similar as pred_boxes except all the correct ones
        iou_threshold (float): threshold where predicted bboxes is correct
        box_format (str): "midpoint" or "corners" used to specify bboxes
        num_classes (int): number of classes

    Returns:
        float: mAP value across all classes given a specific IoU threshold
    """

    # list storing all AP for respective classes
    average_precisions = []

    # used for numerical stability later on
    epsilon = 1e-6

    for c in range(num_classes):
        detections = []
        ground_truths = []

        # Go through all predictions and targets,
        # and only add the ones that belong to the
        # current class c
        for detection in pred_boxes:
            if detection[1] == c:
                detections.append(detection)

        for true_box in true_boxes:
            if true_box[1] == c:
                ground_truths.append(true_box)

        # find the amount of bboxes for each training example
        # Counter here finds how many ground truth bboxes we get
        # for each training example, so let's say img 0 has 3,
        # img 1 has 5 then we will obtain a dictionary with:
        # amount_bboxes = {0:3, 1:5}
        amount_bboxes = Counter([gt[0] for gt in ground_truths])

        # We then go through each key, val in this dictionary
        # and convert to the following (w.r.t same example):
        # ammount_bboxes = {0:torch.tensor[0,0,0], 1:torch.tensor[0,0,0,0,0]}
        for key, val in amount_bboxes.items():
            amount_bboxes[key] = torch.zeros(val)

        # sort by box probabilities which is index 2
        detections.sort(key=lambda x: x[2], reverse=True)
        TP = torch.zeros((len(detections)))
        FP = torch.zeros((len(detections)))
        total_true_bboxes = len(ground_truths)

        # If none exists for this class then we can safely skip
        if total_true_bboxes == 0:
            continue

        for detection_idx, detection in enumerate(detections):
            # Only take out the ground_truths that have the same
            # training idx as detection
            ground_truth_img = [
                bbox for bbox in ground_truths if bbox[0] == detection[0]
            ]

            num_gts = len(ground_truth_img)
            best_iou = 0

            for idx, gt in enumerate(ground_truth_img):
                iou = intersection_over_union(
                    torch.tensor(detection[3:]),
                    torch.tensor(gt[3:]),
                    box_format=box_format,
                )

                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = idx

            if best_iou > iou_threshold:
                # only detect ground truth detection once
                if amount_bboxes[detection[0]][best_gt_idx] == 0:
                    # true positive and add this bounding box to seen
                    TP[detection_idx] = 1
                    amount_bboxes[detection[0]][best_gt_idx] = 1
                else:
                    FP[detection_idx] = 1

            # if IOU is lower then the detection is a false positive
            else:
                FP[detection_idx] = 1

        TP_cumsum = torch.cumsum(TP, dim=0)
        FP_cumsum = torch.cumsum(FP, dim=0)
        recalls = TP_cumsum / (total_true_bboxes + epsilon)
        precisions = TP_cumsum / (TP_cumsum + FP_cumsum + epsilon)
        precisions = torch.cat((torch.tensor([1]), precisions))
        recalls = torch.cat((torch.tensor([0]), recalls))
        # torch.trapz for numerical integration
        average_precisions.append(torch.trapz(precisions, recalls))

    return sum(average_precisions) / len(average_precisions)